In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
# Load tokenizer and model (GPT2 for generation)
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# Load emotion classifier
classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")
def dynamic_hyperparameter_selector(task_type: str):
    if task_type == "factual":
        return {"temperature": 0.3, "top_p": 0.85, "top_k": 40, "repetition_penalty": 1.2}
    elif task_type == "creative":
        return {"temperature": 0.95, "top_p": 0.95, "top_k": 100, "repetition_penalty": 1.0}
    elif task_type == "emotional":
        return {"temperature": 0.7, "top_p": 0.9, "top_k": 50, "repetition_penalty": 1.1}
    else:
        return {"temperature": 0.5, "top_p": 0.85, "top_k": 60, "repetition_penalty": 1.1}

In [ ]:
def infer_task_type(prompt):
    result = classifier(prompt)[0]['label']
    if result in ["joy", "surprise"]:
        return "creative"
    elif result in ["sadness", "fear", "love"]:
        return "emotional"
    else:
        return "factual"

In [ ]:
def generate_response(prompt: str):
    task_type = infer_task_type(prompt)
    hyperparams = dynamic_hyperparameter_selector(task_type)
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        do_sample=True,
        temperature=hyperparams["temperature"],
        top_p=hyperparams["top_p"],
        top_k=hyperparams["top_k"],
        repetition_penalty=hyperparams["repetition_penalty"],
        max_length=200,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True), task_type

In [ ]:
if __name__ == "__main__":
    prompt = input("Enter your prompt: ")
    response, task_type = generate_response(prompt)
    print(f"\n[Task Type: {task_type}]\n")
    print(response)

# Anton 06/19

In [3]:


from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import collections 
from chromaDB_logger import store_to_chroma

# --- Configuration ---
MEMORY_SIZE = 3 # How many past sets of hyperparameters to remember
CURRENT_TASK_WEIGHT = 0.7 # How much the current task type influences the new hyperparameters
PREVIOUS_MEMORY_WEIGHT = 0.3 # How much the average of past hyperparameters influences the new ones

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

# Initialize hyperparameter memory using a deque for fixed-size history
hyperparameter_memory = collections.deque(maxlen=MEMORY_SIZE)

# Vishwa's Function
def dynamic_hyperparameter_selector(task_type: str):
    """
    Selects a base set of hyperparameters based on the task type.
    These are the 'target' hyperparameters for the current prompt.
    """
    if task_type == "factual":
        return {"temperature": 0.3, "top_p": 0.85, "top_k": 40, "repetition_penalty": 1.2}
    elif task_type == "creative":
        return {"temperature": 0.95, "top_p": 0.95, "top_k": 100, "repetition_penalty": 1.0}
    elif task_type == "emotional":
        return {"temperature": 0.7, "top_p": 0.9, "top_k": 50, "repetition_penalty": 1.1}
    else: 
        return {"temperature": 0.5, "top_p": 0.85, "top_k": 60, "repetition_penalty": 1.1}

def infer_task_type(prompt: str) -> str:
    """
    Infers the task type based on the prompt's emotional content.
    """
    result = classifier(prompt)[0]['label']
    if result in ["joy", "surprise"]:
        return "creative"
    elif result in ["sadness", "fear", "love"]:
        return "emotional"
    else:
        return "factual"

# Hyper Parameter Smoothing function -- Anton
def smooth_hyperparameters(current_task_params: dict, memory: collections.deque) -> dict:
    
    if not memory:
        print("Memory is empty. Using direct task parameters.")
        return current_task_params

    avg_memory_params = {
        "temperature": 0.0,
        "top_p": 0.0,
        "top_k": 0.0,
        "repetition_penalty": 0.0
    }

    for past_params in memory:
        for key in avg_memory_params:
            avg_memory_params[key] += past_params[key]

    for key in avg_memory_params:
        avg_memory_params[key] /= len(memory)

    # Apply the weighted average to smooth the parameters
    smoothed_params = {
        "temperature": (current_task_params["temperature"] * CURRENT_TASK_WEIGHT) +
                       (avg_memory_params["temperature"] * PREVIOUS_MEMORY_WEIGHT),
        "top_p": (current_task_params["top_p"] * CURRENT_TASK_WEIGHT) +
                 (avg_memory_params["top_p"] * PREVIOUS_MEMORY_WEIGHT),
        "top_k": int((current_task_params["top_k"] * CURRENT_TASK_WEIGHT) +
                     (avg_memory_params["top_k"] * PREVIOUS_MEMORY_WEIGHT)), 
        "repetition_penalty": (current_task_params["repetition_penalty"] * CURRENT_TASK_WEIGHT) +
                              (avg_memory_params["repetition_penalty"] * PREVIOUS_MEMORY_WEIGHT)
    }

    # Add constraints to ensure parameters stay within reasonable bounds
    smoothed_params["temperature"] = max(0.1, min(smoothed_params["temperature"], 1.0))
    smoothed_params["top_p"] = max(0.1, min(smoothed_params["top_p"], 1.0))
    smoothed_params["top_k"] = max(1, min(smoothed_params["top_k"], tokenizer.vocab_size - 1)) # top_k can't exceed vocab size
    smoothed_params["repetition_penalty"] = max(1.0, min(smoothed_params["repetition_penalty"], 2.0)) # A common range

    return smoothed_params

def generate_response(prompt: str):
    global hyperparameter_memory # Declare intent to modify the global variable

    task_type = infer_task_type(prompt)
    
    # Get the target hyperparameters for the current task
    current_task_target_params = dynamic_hyperparameter_selector(task_type)

    # Smooth these target parameters with the memory
    final_hyperparams = smooth_hyperparameters(current_task_target_params, hyperparameter_memory)

    hyperparameter_memory.append(final_hyperparams)

    print(f"\nDebug Info:")
    print(f"  Inferred Task Type: {task_type}")
    print(f"  Current Task Target Hyperparams: {current_task_target_params}")
    print(f"  Smoothed/Final Hyperparams Used: {final_hyperparams}")
    print(f"  Current Memory State ({len(hyperparameter_memory)} entries): {list(hyperparameter_memory)}")

    # updating the model to bypass the vague response generation Issue - Anton
    new_prompt = "Assume that you are an Ai agent Answering a question\n Question: \" "+prompt +"\""
    inputs = tokenizer(new_prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        do_sample=True,
        temperature=final_hyperparams["temperature"],
        top_p=final_hyperparams["top_p"],
        top_k=final_hyperparams["top_k"],
        repetition_penalty=final_hyperparams["repetition_penalty"],
        max_length=200,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True), task_type

if __name__ == "__main__":
    print("Welcome to the Dynamic Hyperparameter AI Chatbot!")
    print("Type 'quit' or 'exit' to end the conversation.")

    # starting with a 'neutral' base:
    initial_params = {"temperature": 0.5, "top_p": 0.85, "top_k": 60, "repetition_penalty": 1.1}
    hyperparameter_memory.append(initial_params)
    print(f"Initial Memory State: {list(hyperparameter_memory)}")


    while True:
        prompt = input("\nEnter your prompt: ")
        if prompt.lower() in ['quit', 'exit']:
            print("Exiting chat. Goodbye!")
            break

        response, task_type = generate_response(prompt)
        print(f"\n[Inferred Task Type: {task_type}]")
        print(f"Response: {response}")

        # Log this interaction in ChromaDB
        store_to_chroma(prompt, response, task_type)
        

ModuleNotFoundError: No module named 'transformers'